<a href="https://colab.research.google.com/github/reckoner-david/CourseProject/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [1]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import sklearn

In [3]:
df = pd.read_json('train.jsonl' , lines=True)

If the above cell does not work. You will have to add the train and test files. Google Colab does not save files. 





In [7]:
def split_context(arc, level=0):
    return arc[level]

In [11]:
def label_convert(arc):
    if arc=='SARCASM':
        return 0
    if arc=='NOT_SARCASM':
        return 1
    return -1

In [12]:
df['context_1'] = df['context'].apply(lambda x: split_context(x, 0))
df['context_2'] = df['context'].apply(lambda x: split_context(x, 1))
df['sent'] =  df['response'] 

In [13]:
# Converting the label to 0 or 1. 
df['label1'] = df['label'].apply(lambda x: label_convert(x))

In [15]:
# shuffling the data
df = df.sample(frac=1)

In [16]:
train_df = df[['sent' , 'label1']]

In [19]:
train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 1,
    'fp16': False, 
    'sliding_window': False, 
    'learning_rate': 3e-05,
    'max_seq_length': 256,
    'do_lower_case': True,
    'train_batch_size': 4,
    'use_cude': False
}

In [21]:
# use_cuda is false. Enable this for faster training
model = ClassificationModel('distilbert', 'distilbert-base-uncased', args=train_args, use_cuda=False)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [22]:
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:387: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


KeyboardInterrupt: ignored